In [ ]:
import tensorflow as tf
import numpy as np
import math
from tqdm import tqdm

In [ ]:
sess = tf.InteractiveSession()

In [ ]:
#some tensor operation used in neural network
x1 = tf.Variable(tf.truncated_normal([5], mean = 3, stddev = 1./ math.sqrt(5)))
x2 = tf.Variable(tf.truncated_normal([5], mean = 1, stddev = 1./ math.sqrt(5)))
x3 = tf.Variable(tf.truncated_normal([5], mean = 0, stddev = 1./ math.sqrt(5)))

sess.run(tf.initialize_all_variables())

sqx2 = x2 * x2
print(x2.eval())

logx1 = tf.log(x1)
print(x1.eval())
print(logx1.eval())

In [ ]:
sigx3 = tf.sigmoid(x3)
print(x3.eval())
print(sigx3.eval())

In [ ]:
w1 = tf.constant(0.1)
w2 = tf.constant(0.2)
sess.run(tf.initialize_all_variables())

n1 = tf.sigmoid(w1*x1 + w2*x2)
print((w1*x1).eval())
print((w2*x2).eval())
print(n1.eval())

In [ ]:
#Let's build a simple NN

#Load data
data = np.load('data_with_labels.npz')
train = data['arr_0']/255.
labels = data['arr_1']

# Look at some data
print(train[0])
print(labels[0])

# If you have matplotlib installed
import matplotlib.pyplot as plt
plt.ion()

def to_onehot(labels,nclasses = 5):
    '''
    Convert labels to "one-hot" format.

    >>> a = [0,1,2,3]
    >>> to_onehot(a,5)
    array([[ 1.,  0.,  0.,  0.,  0.],
           [ 0.,  1.,  0.,  0.,  0.],
           [ 0.,  0.,  1.,  0.,  0.],
           [ 0.,  0.,  0.,  1.,  0.]])
    '''
    outlabels = np.zeros((len(labels),nclasses))
    for i,l in enumerate(labels):
        outlabels[i,l] = 1
    return outlabels

onehot = to_onehot(labels)

# Split data into training and validation
indices = np.random.permutation(train.shape[0])
valid_cnt = int(train.shape[0] * 0.1)
test_idx, training_idx = indices[:valid_cnt], indices[valid_cnt:]
test, train = train[test_idx,:], train[training_idx,:]
onehot_test, onehot_train = onehot[test_idx,:], onehot[training_idx,:]

sess = tf.InteractiveSession()


# These will be inputs
## Input pixels, flattened
x = tf.placeholder("float", [None, 1296])
## Known labels
y_ = tf.placeholder("float", [None,5])
num_hidden = 128
W1 = tf.Variable(tf.truncated_normal([1296, num_hidden], stddev = 1./math.sqrt(1296)))
b1 = tf.Variable(tf.constant(0.1, shape=[num_hidden]))

h1 = tf.sigmoid(tf.matmul(x, W1) + b1)

W2 = tf.Variable(tf.truncated_normal([num_hidden, 5], stddev = 1./math.sqrt(5)))
b2 = tf.Variable(tf.constant(0.1, shape = [5]))

sess.run(tf.initialize_all_variables())

y = tf.nn.softmax(tf.matmul(h1, W2) + b2)

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y + 1e-20, y_))
                               
#How we train
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

#Define accuracy
                               
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

#End of model build and execute actual training
                               
epochs = 10000
train_acc = np.zeros(epochs//10)
test_acc = np.zeros(epochs//10)
                               
for i in tqdm(range(epochs)):
    if i % 10 == 0:
        A = accuracy.eval(feed_dict={x: train.reshape([-1, 1296]), y_: onehot_train})                       
        train_acc[i//10] = A
        A = accuracy.eval(feed_dict={x: test.reshape([-1, 1296]), y_: onehot_test})                       
        test_acc[i//10] = A
    train_step.run(feed_dict={x: train.reshape([-1, 1296]), y_: onehot_train})

#Plot the accuracy curve
plt.plot(train_acc, 'bo')
plt.plot(test_acc, 'rx')
                               
#Look at the final testing confusion matrix
pred = np.argmax(y.eval(feed_dict={x: test.reshape([-1, 1296]), y_: onehot_test}), axis = 1)
conf = np.zeros([5,5])
for p, t in zip(pred,np.argmax(onehot_test, axis=1)):
    conf[t,p] += 1
                               
plt.matshow(conf)
plt.colorbar()       

In [ ]:
#let's look at subplot of some weights
f, plts = plt.subplots(4,8,sharex=True)
for i in range(32):
    plts[i//8, i%8].pcolormesh(W1.eval()[:, i].reshape([36,36]))